In [8]:
import pandas as pd

In [9]:
import os
dir = os.path.dirname(os.path.dirname(os.getcwd()))
print(dir)

c:\Users\thsat\sat_imagery_ml\sat-imagery-ml-1


In [10]:
path_eruptions_preprocessed = os.path.join(dir, "Eruption_data", "eruptions_preprocessed.csv")
df_eruptions = pd.read_csv(path_eruptions_preprocessed)
# df_eruptions.head()

In [11]:
df_eruptions['Eruption Number'].unique()

array([22609, 22613, 22608, 22610, 22612, 22606, 22607, 22611, 22604,
       22601, 22602, 22603, 22605, 22597, 22600, 22593, 22591, 22592,
       22594, 22596, 22598, 22587, 22588, 22595, 22582, 22580, 22581,
       22579, 22566, 22567, 22589, 22568, 22569, 22573, 22570, 22571,
       22572, 22562, 22563, 22564, 22561, 22560, 22554, 22557, 22556,
       22558, 22553, 22555, 22549, 22550, 22559, 22552, 22551, 22548,
       22547, 22541, 22536, 22535, 22539, 22543, 22540, 22534, 22538,
       22542, 22521, 22508, 22499, 22577, 22500, 22503, 22502, 22501,
       22493, 22494, 22492, 22489, 22490, 22491, 22486, 22479, 22484,
       22481, 22482, 22477, 22478, 22485, 22483, 22459, 22458, 22455,
       22457, 22480, 22450, 22453, 22446, 22454, 22445, 22448, 22451,
       22447, 22586, 22434, 22430, 22456, 22431, 22396, 22429, 22420,
       22423, 22421, 22475, 22452, 22425, 22575, 22424, 22416, 22417,
       22407, 22408, 22522, 22410, 22411, 22409, 22405, 22414, 22413,
       22406, 22412,

In [12]:
# unique_volcano_names = df_eruptions['Volcano Name'].unique()
# print(unique_volcano_names)

In [13]:
import numpy as np

import ee
ee.Initialize(project='bustling-syntax-392010')
print('done')

# parse and reorder bbox column to correct coordinate format for Earth Engine
def parse_bbox_string(bbox_str):
    try:
        coords = list(map(float, bbox_str.strip('[] ').split(',')))
        if len(coords) != 4:
            raise ValueError
        lat1, lon1, lat2, lon2 = coords
        return ee.Geometry.BBox(lon1, lat1, lon2, lat2)
    except Exception:
        return np.nan  # return NaN for invalid rows

df_eruptions['bbox'] = df_eruptions['bbox'].astype(str)

df_eruptions['ee_bbox'] = df_eruptions['bbox'].apply(parse_bbox_string)
df_eruptions = df_eruptions.dropna(subset=['ee_bbox'])

done


In [14]:
targets = ["300250", "243080","260010", "273030", "263250", "341090", "211060", "211040", "332020", "223030", "332010"]
df_eruptions = df_eruptions[df_eruptions['Volcano Number'].astype(str).isin(targets)]

df_eruptions = df_eruptions[df_eruptions['Start Year'] >= 2016]

In [15]:
print(df_eruptions['Start_Date'].describe())
print("\nUnique Start Dates:", df_eruptions['Start_Date'].nunique())
print("\nDate Range:", df_eruptions['Start_Date'].min(), "to", df_eruptions['Start_Date'].max())

count             27
unique            23
top       2022-11-27
freq               3
Name: Start_Date, dtype: object

Unique Start Dates: 23

Date Range: 2016-12-05 to 2024-12-24


In [25]:
df_22593_Karymsky = df_eruptions[df_eruptions['Eruption Number'] == 22581]
df_22593_Karymsky


,Volcano Number,Eruption Number,Volcano Name,Start Year Modifier,Start Year,Start Year Uncertainty,Start Month,Start Day Modifier,Start Day,Start Day Uncertainty,...,End Month,End Day Modifier,End Day,End Day Uncertainty,Latitude,Longitude,Start_Date,End_Date,bbox,ee_bbox


In [17]:
df_22593_Raung = df_eruptions[df_eruptions['Eruption Number'] == 22593]
df_22593_Raung

,Volcano Number,Eruption Number,Volcano Name,Start Year Modifier,Start Year,Start Year Uncertainty,Start Month,Start Day Modifier,Start Day,Start Day Uncertainty,...,End Month,End Day Modifier,End Day,End Day Uncertainty,Latitude,Longitude,Start_Date,End_Date,bbox,ee_bbox


In [35]:
df_22608_bulusan = df_eruptions[df_eruptions['Eruption Number'] == 22608]
df_22608_bulusan

,Volcano Number,Eruption Number,Volcano Name,Start Year Modifier,Start Year,Start Year Uncertainty,Start Month,Start Day Modifier,Start Day,Start Day Uncertainty,...,End Month,End Day Modifier,End Day,End Day Uncertainty,Latitude,Longitude,Start_Date,End_Date,bbox,ee_bbox


# Image config

# Get data

In [16]:
# False-color viz parameters (for the lava flow) with increased contrast
false_color_vis = {
    'bands': ['B12', 'B11', 'B8A'],
    'min': 0,
    'max': 4000,  # increase max value for higher contrast
    'gamma': 0.8  # Lower gamma to enhance saturation
}

In [17]:
# Output base directory
output_dir = './sentinel_pngs/'
os.makedirs(output_dir, exist_ok=True)

print("Saving images to:", os.path.abspath(output_dir))

Saving images to: c:\Users\thsat\sat_imagery_ml\sat-imagery-ml-1\1_DatasetCharacteristics\EruptionImages\sentinel_pngs


# YesActivity // NoActivity

In [38]:
import os
import requests
import pandas as pd
import ee
from datetime import timedelta


def download_false_color_series(row, duration_days: int, cloudy_pixel_percentage: int, output_dir: str, false_color_vis: dict):
    eruption_id = row['Volcano Name'].replace(' ', '_') + "_" + str(row['Eruption Number'])
    start_date = pd.to_datetime(row['Start_Date'])
    bbox = row['ee_bbox']  # Must be ee.Geometry.BBox or Polygon

    start_window = (start_date - timedelta(days=duration_days)).strftime('%Y-%m-%d')
    end_window = (start_date + timedelta(days=duration_days)).strftime('%Y-%m-%d')

    s2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
        .filterDate(start_window, end_window) \
        .filterBounds(bbox) \
        .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', cloudy_pixel_percentage)

    try:
        s2_list = s2.toList(s2.size())
        count = s2.size().getInfo()

        if count == 0:
            print(f"No images found for {eruption_id} in the specified range.")
            return

        # Create base directory for eruption
        eruption_dir = os.path.join(output_dir, eruption_id)
        no_activity_dir = os.path.join(eruption_dir, "NoActivity")
        yes_activity_dir = os.path.join(eruption_dir, "YesActivity")
        os.makedirs(no_activity_dir, exist_ok=True)
        os.makedirs(yes_activity_dir, exist_ok=True)

        for i in range(0, count):
            try:
                img = ee.Image(s2_list.get(i))
                date_str = img.date().format('YYYY-MM-dd').getInfo()
                img_date = pd.to_datetime(date_str)

                # Decide subfolder based on image date
                subfolder = no_activity_dir if img_date < start_date else yes_activity_dir

                # Select and visualize bands
                raw_img = img.clip(bbox).select(['B12', 'B11', 'B8A'])
                vis_img = raw_img.visualize(**false_color_vis)

                url = vis_img.getThumbURL({
                    'region': bbox.getInfo(),
                    'dimensions': 512,
                    'format': 'png'
                })

                file_path = os.path.join(subfolder, f'{eruption_id}_{date_str}.png')

                response = requests.get(url)
                if response.status_code == 200:
                    with open(file_path, 'wb') as f:
                        f.write(response.content)
                    print(f"Downloaded: {file_path}")
                else:
                    print(f"Failed ({response.status_code}) for {eruption_id} on {date_str}")

            except Exception as e:
                print(f"Error processing image {i} for {eruption_id}: {e}")

    except Exception as e:
        print(f"Failed to process {eruption_id}: {e}")


In [39]:
# Example loop over a DataFrame containing eruption data
for _, row in df_eruptions.iterrows():
    download_false_color_series(row, duration_days=365, cloudy_pixel_percentage=20, output_dir=output_dir, false_color_vis=false_color_vis)

Downloaded: ./sentinel_pngs/Bezymianny_22605\NoActivity\Bezymianny_22605_2023-12-25.png
Downloaded: ./sentinel_pngs/Bezymianny_22605\NoActivity\Bezymianny_22605_2023-12-25.png
Downloaded: ./sentinel_pngs/Bezymianny_22605\NoActivity\Bezymianny_22605_2023-12-25.png
Downloaded: ./sentinel_pngs/Bezymianny_22605\NoActivity\Bezymianny_22605_2023-12-25.png
Downloaded: ./sentinel_pngs/Bezymianny_22605\NoActivity\Bezymianny_22605_2024-01-04.png
Downloaded: ./sentinel_pngs/Bezymianny_22605\NoActivity\Bezymianny_22605_2024-01-04.png
Downloaded: ./sentinel_pngs/Bezymianny_22605\NoActivity\Bezymianny_22605_2024-01-04.png
Downloaded: ./sentinel_pngs/Bezymianny_22605\NoActivity\Bezymianny_22605_2024-01-04.png
Downloaded: ./sentinel_pngs/Bezymianny_22605\NoActivity\Bezymianny_22605_2024-01-07.png
Downloaded: ./sentinel_pngs/Bezymianny_22605\NoActivity\Bezymianny_22605_2024-01-07.png
Downloaded: ./sentinel_pngs/Bezymianny_22605\NoActivity\Bezymianny_22605_2024-01-09.png
Downloaded: ./sentinel_pngs/Bezy

# Specific bbox and date

In [32]:
import os
import requests
import pandas as pd
import ee
from datetime import datetime, timedelta

ee.Initialize(project='bustling-syntax-392010')

def download_false_color_series_bbox(bbox_coords: list, volcano_name: str, volcano_number: int, center_date_str: str,
                                     duration_days: int, cloudy_pixel_percentage: int, output_dir: str,
                                     false_color_vis: dict):
    # format center date
    center_date = pd.to_datetime(center_date_str)

    bbox = ee.Geometry.BBox(*bbox_coords)

    start_window = (center_date - timedelta(days=duration_days)).strftime('%Y-%m-%d')
    end_window = (center_date + timedelta(days=duration_days)).strftime('%Y-%m-%d')

    # get sentilel 2 data
    s2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
        .filterDate(start_window, end_window) \
        .filterBounds(bbox) \
        .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', cloudy_pixel_percentage)

    try:
        s2_list = s2.toList(s2.size())
        count = s2.size().getInfo()

        if count == 0:
            print(f"No images found for {volcano_name} in the specified range.")
            return

        # directories
        volcano_id = f"{volcano_name.replace(' ', '_')}_{volcano_number}"
        volcano_dir = os.path.join(output_dir, volcano_id)
        before_dir = os.path.join(volcano_dir, "NoActivity")
        after_dir = os.path.join(volcano_dir, "YesActivity")
        os.makedirs(before_dir, exist_ok=True)
        os.makedirs(after_dir, exist_ok=True)

        for i in range(count):
            try:
                img = ee.Image(s2_list.get(i))
                date_str = img.date().format('YYYY-MM-dd').getInfo()
                img_date = pd.to_datetime(date_str)

                subfolder = before_dir if img_date < center_date else after_dir

                
                raw_img = img.clip(bbox).select(['B12', 'B11', 'B8A'])
                vis_img = raw_img.visualize(**false_color_vis)

                url = vis_img.getThumbURL({
                    'region': bbox.getInfo(),
                    'dimensions': 512,
                    'format': 'png'
                })

                file_path = os.path.join(subfolder, f'{volcano_id}_{date_str}.png')

                response = requests.get(url)
                if response.status_code == 200:
                    with open(file_path, 'wb') as f:
                        f.write(response.content)
                    print(f"Downloaded: {file_path}")
                else:
                    print(f"Failed ({response.status_code}) on {date_str}")

            except Exception as e:
                print(f"Error processing image {i}: {e}")

    except Exception as e:
        print(f"Failed to process {volcano_name}: {e}")



In [33]:
import os

output_dir = './sentinel_pngs/'
os.makedirs(output_dir, exist_ok=True)
print("Saving images to:", os.path.abspath(output_dir))

false_color_vis = {
    'bands': ['B12', 'B11', 'B8A'],
    'min': 0,
    'max': 4000,  # increase max value for higher contrast
    'gamma': 0.8  # lower gamma to enhance saturation
}

Saving images to: c:\Users\thsat\sat_imagery_ml\sat-imagery-ml-1\1_DatasetCharacteristics\EruptionImages\sentinel_pngs


In [34]:
bbox_info = {
    'Volcano Name': 'Popocatepetl',
    'Volcano Number': 341090,
    'bbox': [-98.642, 19.003, -98.602, 19.043]
}
center_date_str = "2025-01-20"

# bbox_info = {
#     'Volcano Name': 'Stromboli',
#     'Volcano Number': 223030,
#     'bbox': [15.183, 38.759, 15.243, 38.759]
# }
# center_date_str =  "2019-07-12",

# bbox_info = {
#     'Volcano Name': 'Nyiragongo',
#     'Volcano Number': 223030,
#     'bbox': [-1.5284340969712804, 29.24300017519119, -1.515805117200142, 29.255567353688676]
# }

# "2021-05-01",


# bbox_info = {
#     'Volcano Name': 'Home Reef',
#     'Volcano Number': 243080,
#     'bbox': [-174.79, -19.007, -174.76000000000002, -18.977]
# }
# "2024-06-20",


# [-174.79, -19.007], [-174.76000000000002, -18.977] # home reef
# Home Reef
# 243080
# center_date_str="2024-06-20"

download_false_color_series_bbox(
    bbox_coords=bbox_info['bbox'],
    volcano_name=bbox_info['Volcano Name'],
    volcano_number=bbox_info['Volcano Number'],
    center_date_str=center_date_str,
    duration_days=365,
    cloudy_pixel_percentage=20,
    output_dir=output_dir,
    false_color_vis=false_color_vis
)

Downloaded: ./sentinel_pngs/Popocatepetl_341090\NoActivity\Popocatepetl_341090_2024-01-26.png
Downloaded: ./sentinel_pngs/Popocatepetl_341090\NoActivity\Popocatepetl_341090_2024-02-05.png
Downloaded: ./sentinel_pngs/Popocatepetl_341090\NoActivity\Popocatepetl_341090_2024-02-10.png
Downloaded: ./sentinel_pngs/Popocatepetl_341090\NoActivity\Popocatepetl_341090_2024-02-15.png
Downloaded: ./sentinel_pngs/Popocatepetl_341090\NoActivity\Popocatepetl_341090_2024-02-20.png
Downloaded: ./sentinel_pngs/Popocatepetl_341090\NoActivity\Popocatepetl_341090_2024-02-25.png
Downloaded: ./sentinel_pngs/Popocatepetl_341090\NoActivity\Popocatepetl_341090_2024-03-01.png
Downloaded: ./sentinel_pngs/Popocatepetl_341090\NoActivity\Popocatepetl_341090_2024-03-06.png
Downloaded: ./sentinel_pngs/Popocatepetl_341090\NoActivity\Popocatepetl_341090_2024-03-11.png
Downloaded: ./sentinel_pngs/Popocatepetl_341090\NoActivity\Popocatepetl_341090_2024-03-16.png
Downloaded: ./sentinel_pngs/Popocatepetl_341090\NoActivity\P

In [ ]:
footprint = img.geometry().bounds().getInfo()
print(f"Image footprint: {footprint}")

# raw reflectance band as numerical arrays // geotiffs

In [18]:
import os
import requests
import pandas as pd
import ee
from datetime import datetime, timedelta

# Initialize Earth Engine
ee.Initialize(project='bustling-syntax-392010')

def download_raw_bands_as_geotiff(bbox_coords: list, volcano_name: str, volcano_number: int, center_date_str: str,
                                   duration_days: int, cloudy_pixel_percentage: int, output_dir: str):
    # Parse eruption date
    center_date = pd.to_datetime(center_date_str)

    # Define bbox
    bbox = ee.Geometry.BBox(*bbox_coords)

    # Time window
    start_window = (center_date - timedelta(days=duration_days)).strftime('%Y-%m-%d')
    end_window = (center_date + timedelta(days=duration_days)).strftime('%Y-%m-%d')

    # Sentinel-2 collection
    s2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
        .filterDate(start_window, end_window) \
        .filterBounds(bbox) \
        .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', cloudy_pixel_percentage)

    try:
        s2_list = s2.toList(s2.size())
        count = s2.size().getInfo()

        if count == 0:
            print(f"No images found for {volcano_name} in the specified range.")
            return

        # Set up directory
        volcano_id = f"{volcano_name.replace(' ', '_')}_{volcano_number}"
        volcano_dir = os.path.join(output_dir, volcano_id)
        before_dir = os.path.join(volcano_dir, "NoActivity")
        after_dir = os.path.join(volcano_dir, "YesActivity")
        os.makedirs(before_dir, exist_ok=True)
        os.makedirs(after_dir, exist_ok=True)

        for i in range(count):
            try:
                img = ee.Image(s2_list.get(i))
                date_str = img.date().format('YYYY-MM-dd').getInfo()
                img_date = pd.to_datetime(date_str)

                subfolder = before_dir if img_date < center_date else after_dir

                # Clip and select bands (raw reflectance)
                raw_img = img.clip(bbox).select(['B12', 'B11', 'B8A'])

                # Create download URL for GeoTIFF
                url = raw_img.getDownloadURL({
                    'name': f'{volcano_id}_{date_str}',
                    'bands': ['B12', 'B11', 'B8A'],
                    'scale': 20,  # resolution in meters
                    'region': bbox.getInfo(),
                    'format': 'GeoTIFF'
                })

                # Download
                file_path = os.path.join(subfolder, f'{volcano_id}_{date_str}.tif')
                response = requests.get(url)

                if response.status_code == 200:
                    with open(file_path, 'wb') as f:
                        f.write(response.content)
                    print(f"Downloaded GeoTIFF: {file_path}")
                else:
                    print(f"Failed ({response.status_code}) on {date_str}")

            except Exception as e:
                print(f"Error processing image {i}: {e}")

    except Exception as e:
        print(f"Failed to process {volcano_name}: {e}")

In [19]:
# Set up output directory
output_dir = './sentinel_geotiffs/'
os.makedirs(output_dir, exist_ok=True)
print("Saving raw GeoTIFFs to:", os.path.abspath(output_dir))

bbox_info = {
    'Volcano Name': 'Home Reef',
    'Volcano Number': 243080,
    'bbox': [-174.79, -19.007, -174.76000000000002, -18.977]
}

# Call the function
download_raw_bands_as_geotiff(
    bbox_coords=bbox_info['bbox'],
    volcano_name=bbox_info['Volcano Name'],
    volcano_number=bbox_info['Volcano Number'],
    center_date_str="2024-06-20",
    duration_days=365,
    cloudy_pixel_percentage=20,
    output_dir=output_dir
)

Saving raw GeoTIFFs to: c:\Users\thsat\sat_imagery_ml\sat-imagery-ml-1\1_DatasetCharacteristics\EruptionImages\sentinel_geotiffs
Downloaded GeoTIFF: ./sentinel_geotiffs/Home_Reef_243080\NoActivity\Home_Reef_243080_2023-06-21.tif
Downloaded GeoTIFF: ./sentinel_geotiffs/Home_Reef_243080\NoActivity\Home_Reef_243080_2023-06-21.tif
Downloaded GeoTIFF: ./sentinel_geotiffs/Home_Reef_243080\NoActivity\Home_Reef_243080_2023-06-26.tif
Downloaded GeoTIFF: ./sentinel_geotiffs/Home_Reef_243080\NoActivity\Home_Reef_243080_2023-06-26.tif
Downloaded GeoTIFF: ./sentinel_geotiffs/Home_Reef_243080\NoActivity\Home_Reef_243080_2023-07-06.tif
Downloaded GeoTIFF: ./sentinel_geotiffs/Home_Reef_243080\NoActivity\Home_Reef_243080_2023-07-06.tif
Downloaded GeoTIFF: ./sentinel_geotiffs/Home_Reef_243080\NoActivity\Home_Reef_243080_2023-07-21.tif
Downloaded GeoTIFF: ./sentinel_geotiffs/Home_Reef_243080\NoActivity\Home_Reef_243080_2023-07-21.tif
Downloaded GeoTIFF: ./sentinel_geotiffs/Home_Reef_243080\NoActivity\Hom

KeyboardInterrupt: 

In [21]:
import rasterio
#1_DatasetCharacteristics\EruptionImages\sentinel_geotiffs\Home_Reef_243080\NoActivity\Home_Reef_243080_2023-06-21.tif
path = r"C:\Users\thsat\sat_imagery_ml\sat-imagery-ml-1\1_DatasetCharacteristics\EruptionImages\sentinel_geotiffs\Home_Reef_243080\NoActivity\Home_Reef_243080_2024-06-15.tif"

with rasterio.open(path) as src:
    print("CRS:", src.crs)
    print("Width:", src.width)
    print("Height:", src.height)
    print("Number of bands:", src.count)
    print("Data type:", src.dtypes[0])
    
    band12 = src.read(1)
    band11 = src.read(2)
    band8a = src.read(3)
    
    print("B12 (SWIR2) sample values:", band12[0:5, 0:5])


CRS: EPSG:32701
Width: 161
Height: 169
Number of bands: 3
Data type: uint16
B12 (SWIR2) sample values: [[ 0  0 16 14 15]
 [ 0  0 10 17 14]
 [ 0  0 18 20 22]
 [ 0  0 17 20 15]
 [ 0  0 16 20 19]]


In [28]:
band11

array([[ 0,  0, 25, ...,  0,  0,  0],
       [ 0,  0, 22, ...,  0,  0,  0],
       [ 0,  0, 31, ..., 48, 34, 38],
       ...,
       [31, 32, 31, ..., 37,  0,  0],
       [ 0,  0,  0, ..., 41,  0,  0],
       [ 0,  0,  0, ..., 43,  0,  0]], shape=(169, 161), dtype=uint16)

# archive

In [ ]:
# # without yesactivity noactivity subfolders

# import requests
# from datetime import timedelta


# def download_false_color_series(row, duration_days: int, cloudy_pixel_percentage: int):
#     eruption_id = row['Volcano Name'].replace(' ', '_') + "_" + str(row['Eruption Number'])
#     start_date = pd.to_datetime(row['Start_Date'])
#     bbox = row['ee_bbox']  # Must be ee.Geometry.BBox or Polygon

#     start_window = (start_date - timedelta(days=duration_days)).strftime('%Y-%m-%d')
#     end_window = (start_date + timedelta(days=duration_days)).strftime('%Y-%m-%d')

#     s2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
#         .filterDate(start_window, end_window) \
#         .filterBounds(bbox) \
#         .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', cloudy_pixel_percentage)

#     try:
#         s2_list = s2.toList(s2.size())
#         count = s2.size().getInfo()

#         if count == 0:
#             print(f"No images found for {eruption_id} in the specified range.")
#             return

#         eruption_dir = os.path.join(output_dir, eruption_id)
#         os.makedirs(eruption_dir, exist_ok=True)


#         for i in range(0, count):
#             try:
#                 img = ee.Image(s2_list.get(i))
#                 date_str = img.date().format('YYYY-MM-dd').getInfo()

#                 # select raw bands before visualizing
#                 raw_img = img.clip(bbox).select(['B12', 'B11', 'B8A'])
#                 #print("Raw bands:", raw_img.bandNames().getInfo())

#                 vis_img = raw_img.visualize(**false_color_vis)
#                 #print("selected bands:", vis_img.bandNames().getInfo())

#                 url = vis_img.getThumbURL({
#                     'region': bbox.getInfo(),
#                     'dimensions': 512,
#                     'format': 'png'
#                 })

#                 file_path = os.path.join(eruption_dir, f'{eruption_id}_{date_str}.png')

#                 response = requests.get(url)
#                 if response.status_code == 200:
#                     with open(file_path, 'wb') as f:
#                         f.write(response.content)
#                     print(f"Downloaded: {file_path}")
#                 else:
#                     print(f"Failed ({response.status_code}) for {eruption_id} on {date_str}")

#             except Exception as e:
#                 print(f"Error processing image {i} for {eruption_id}: {e}")

#     except Exception as e:
#         print(f"Failed to process {eruption_id}: {e}")

In [ ]:
# # Example loop over a DataFrame containing eruption data
# for _, row in df_eruptions.iterrows():
#     download_false_color_series(row, duration_days=365, cloudy_pixel_percentage=20)